In [1]:
import originpro as op
import os
import numpy as np
import pandas as pd

In [2]:
#Template Path
template = os.path.normpath(r'C:\Users\ajinkya\Desktop\Kachra\150721\ImporttoFFT.otwu')
formula  = '(2000+ (i-1)*(17000-2000)/V1)/149.896229'


datapath = os.path.normpath(r'C:\Users\ajinkya\Desktop\Kachra\150721')
date = '150721'
run = [4,5,7,8]
total_files = [5,5,5,5]

In [3]:
#My Functions
def readfile(filename):
    import pandas as pd
    import numpy as np
    
    df = pd.read_csv(filename,skiprows=12,skipfooter=1,sep='\s+').iloc[:-1,:5]
    df=pd.DataFrame(np.array(df, dtype=float))
    index=["Distance (um)", "Delay (ps)", "Signal (V)", "Phase (deg)", "Rcos (theta)"]
    df=df.set_axis(index, axis=1, inplace=False).reset_index(drop=True)
    return df

def run_numbering(i):
    if i<10:
        temp_name = f'00%i'%i
    elif i>=100:
        temp_name = f'%i'%i
    else:
        temp_name = f'0%i'%i
    return temp_name

def filename_generator(date, run, total_files):
    i = 0
    filenames = []
    for run_ in run:
        for total_files_ in range(total_files[i]):
            #filename = date + '_' + str(run_numbering(run_)) + 'forward_proct_' + str(total_files_) + '.txt'
            filename = date + '_' + str(run_numbering(run_)) + '_br_' + str(total_files_) + '.txt'
            filenames.append(filename)
            filename = date + '_' + str(run_numbering(run_)) + '_ft_' + str(total_files_) + '.txt'
            filenames.append(filename)
        i = i+1
    return filenames

#Reference: https://www.techiedelight.com/convert-given-number-corresponding-excel-column-name/
# Function to convert a given number to an Excel column
def getColumnName(n):
 
    # initialize output string as empty
    result = ""
 
    while n > 0:
 
        # find the index of the next letter and concatenate the letter
        # to the solution
 
        # here index 0 corresponds to `A`, and 25 corresponds to `Z`
        index = (n - 1) % 26
        result += chr(index + ord('A'))
        n = (n - 1) // 26
 
    return result[::-1]

In [4]:
# Very useful, especially during development, when you are
# liable to have a few uncaught exceptions.
# Ensures that the Origin instance gets shut down properly.
# Note: only applicable to external Python.
import sys
def origin_shutdown_exception_hook(exctype, value, traceback):
    '''Ensures Origin gets shut down if an uncaught exception'''
    op.exit()
    sys.__excepthook__(exctype, value, traceback)
if op and op.oext:
    sys.excepthook = origin_shutdown_exception_hook

In [5]:
# Set Origin instance visibility.
# Important for only external Python.
# Should not be used with embedded Python. 
if op.oext:
    op.set_show(True)

In [6]:
#filepath = filename_generator(date, run, total_files)
#np.savetxt('analysisfilename.txt',filepath,delimiter=',',fmt='%s')

#df = pd.read_csv('analysisfilename.txt')
df = pd.read_csv('analysisfilename.txt',header=None,sep='\t').to_numpy()
filepath = df[:,0].tolist()
comment  = df[:,1].tolist()

In [12]:
amplitude     = op.new_sheet(lname='Amplitude')
phase         = op.new_sheet(lname='Phase')
delay         = op.new_sheet(lname='Delay')
transmittance = op.new_sheet(lname='Transmittance')
phasediff     = op.new_sheet(lname='Phasediff')

delay_1 = op.new_sheet(lname='Delay2')

#Set Frequency Column
amplitude.set_label('A','Frequency','L')
amplitude.set_label('A','THz','U')

phase.set_formula('A','[Amplitude]Sheet1!A')
phase.set_label('A','Frequency','L')
phase.set_label('A','THz','U')

delay.set_label('A','Frequency','L')
delay.set_label('A','THz','U')

In [16]:
increment = 2
maxindex = []
maxindexvalue = []
for filepath_ in filepath:     
       
    print(filepath_)
    #op.new_book(type = _WORKBOOK, lname='', template='',hidden=False)
    newbook = op.new_book('w',filepath_,template,False)

    #Activate the template sheet
    activesheet = op.find_sheet()

    #read the file from where the data is to imported
    df = readfile(os.path.normpath(datapath + '\\' + filepath_))

    #Place the data into the worksheet
    activesheet.from_list(0, df.iloc[:,0])
    activesheet.from_list(1, df.iloc[:,1])
    activesheet.from_list(2, df.iloc[:,2])
    activesheet.from_list(3, df.iloc[:,3])
    activesheet.from_list(4, df.iloc[:,4])
    
    maxindex.append(df.iloc[np.argmax(df.iloc[:,4]),:].to_numpy().tolist())
    maxindexvalue.append(np.argmax(df.iloc[:,4]))
    
    #Frequency column in Amplitude worksheet and Delay column in Delay worksheet
    if filepath_ == filepath[0]:
        amplitude.set_formula('A','[' + filepath_ +  ']Sheet1!L')
        delay.set_formula('A','[' + filepath_ +  ']Sheet1!J')
        
    col = getColumnName(increment)
    comments = comment[increment-2]
    
    amplitude.set_formula(col,'[' + filepath_ +  ']Sheet1!Q')
    amplitude.set_label(col,'Amplitude','L')
    amplitude.set_label(col,'a.u.','U')
    amplitude.set_label(col,comments,'C')
    
    phase.set_formula(col,'[' + filepath_ +  ']Sheet1!R')
    phase.set_label(col,'Phase','L')
    phase.set_label(col,'degree','U')
    phase.set_label(col,comments,'C')
    
    delay.set_formula(col,'[' + filepath_ +  ']Sheet1!K')
    delay.set_label(col,'Amplitude','L')
    delay.set_label(col,'ps','U')
    delay.set_label(col,comments,'C')
    
    #delay_1.set_formula(getColumnName((increment-2)*2+1),'[' + filepath_ +  ']Sheet1!A')
    #delay_1.set_label(getColumnName((increment-2)*2+1),'Amplitude','L')
    #delay_1.set_label(getColumnName((increment-2)*2+1),'ps','U')
    #delay_1.set_label(getColumnName((increment-2)*2+1),comments,'C')
    
    #delay_1.set_formula(getColumnName((increment-1)*2),'[' + filepath_ +  ']Sheet1!E')
    #delay_1.set_label(getColumnName((increment-1)*2),'Amplitude','L')
    #delay_1.set_label(getColumnName((increment-1)*2),'ps','U')
    #delay_1.set_label(getColumnName((increment-1)*2),comments,'C')
    
    #Set Formula for delay 'um' to delay 'ps'
    #activesheet.set_formula('G',formula)
    
    increment = increment + 1

150721.009


<ipython-input-3-d3fad2520f45>:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(filename,skiprows=12,skipfooter=1,sep='\s+').iloc[:-1,:5]


150721_004_br_0.txt
150721_004_ft_0.txt
150721_004_br_1.txt
150721_004_ft_1.txt
150721_004_br_2.txt
150721_004_ft_2.txt
150721_004_br_3.txt
150721_004_ft_3.txt
150721_005_br_0.txt
150721_005_ft_0.txt
150721_005_br_1.txt
150721_005_ft_1.txt
150721_005_br_2.txt
150721_005_ft_2.txt
150721_005_br_3.txt
150721_005_ft_3.txt
150721_007_br_0.txt
150721_007_ft_0.txt
150721_007_br_1.txt
150721_007_ft_1.txt
150721_007_br_2.txt
150721_007_ft_2.txt
150721_007_br_3.txt
150721_007_ft_3.txt
150721_008_br_0.txt
150721_008_ft_0.txt
150721_008_br_1.txt
150721_008_ft_1.txt
150721_008_br_2.txt
150721_008_ft_2.txt
150721_008_br_3.txt
150721_008_ft_3.txt


In [17]:
# Exit running instance of Origin.
# Required for external Python but don't use with embedded Python.
if op.oext:
    op.exit()

In [ ]:
#np.savetxt('peak.txt',maxindex,delimiter=',',fmt='%s')

In [ ]:
#np.savetxt('peak_.txt',maxindexvalue,delimiter=',',fmt='%s')